# Find Artist

## 개요

### 실습 목표

* 추천 시스템의 개념과 목적을 이해한다.
* Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
* 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
* 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
* 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
* 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.

### 목차

1. 추천 시스템이란 게 뭔가요?
2. 데이터 탐색하기와 전처리
3. 사용자의 명시적/암묵적 평가
4. Matrix Factorization(MF)
5. CSR(Compressed Sparse Row) Matrix
6. MF 모델 학습하기
7. 비슷한 아티스트 찾기 + 유저에게 추천하기
8. 프로젝트 - MovieLens 영화 추천 실습

## 추천시스템이란

* 아이템은 많고 유저 취향이 다양할 때, 유저가 소비할 만한 아이템을 예측하는 모델입니다.

## 데이터 탐색하기와 전처리

### 데이터 준비

* tsv (Tab-Separated Values): csv(Comma-Separated Values)와 유사하나 구분자만 tab을 활용한 것

In [1]:
import pandas as pd
import os

fname = 'data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


* artist_MBID 는 우리의 관심사가 아닙니다. 사용하는 컬럼만 남기고 제거하겠습니다.

In [2]:
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


* 용이한 검색을 위해 소문자로 변환하겠습니다.

In [3]:
data['artist'] = data['artist'].str.lower() # 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


* 첫 번째 유저가 어떤 노래를 듣는지 확인해봅시다.

In [4]:
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


### 데이터 탐색

* 유저 수, 아티스트 수, 인기 많은 아티스트
* 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
* 유저 play 횟수 중앙값에 대한 통계

In [5]:
print('User 수 :', data['user_id'].nunique())
print('Artist 수:', data['artist'].nunique())

User 수 : 358868
Artist 수: 291346


In [6]:
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [7]:
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [8]:
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

### 사용자 초기 정보 세팅 (모델 검증용)

In [9]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                        # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

/tmp/ipykernel_15950/1252709198.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다.


,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


### 모델을 위한 전처리

In [10]:
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

* 유니크한 것들 중에 마지막에 zimin이 추가되었으니 총 358869 명의 유저 중 마지막이니 358868

In [11]:
print(user_to_idx['zimin'])
print(artist_to_idx['black eyed peas'])

358868
376


* indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
* dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.
* user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
* 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다.
* artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 

In [12]:
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data 
else:
    print('user_id column indexing Fail!!')

temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


## 명시적/ 암묵적 평가

* 사용자의 아이템 선호도를 말해 주는 유저 행동 데이터셋이 필요합니다.
* 가지고 있는 데이터는 평점이 아닌 몇 번 플레이 했나 뿐
* 자연스럽게 발생하는 암묵적(implicit) 피드백도 사용자의 아이템에 대한 평가를 알 수 있는 단서가 된다.

* 부정적인 피드백이 없다.(No Negative Feedback)
* 애초에 잡음이 많다.(Inherently Noisy)
* 수치는 신뢰도를 의미한다.(The numerical value of implicit feedback indicates confidence)
* Implicit-feedback Recommender System의 평가는 적절한 방법을 고민해봐야 한다.(Evaluation of implicit-feedback recommender requires appropriate measures)

* 1회만 플레이한 데이터의 비율
* 한번이라도 들었으면 선호한다고 판단
* 많이 재생한 아티스트에 대해 가중치를 준다.
* f-format에 대한 설명은 https://bit.ly/2DTLqYU

In [13]:
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')

147740,17535660
Ratio of only_one over all data is 0.84%


## Matrix Factorization(MF)

* m명의 사용자들과 n명의 아티스트에 대한 평가 데이터가 포함한 (m,n)사이즈의 평가 행렬을 만들 것 입니다.
* 협업 필터링 : 이 평가 행렬을 전제로 하는 것
* Matrix Factorization(MF, 행렬 분해) 모델을 사용
* 큰 평가 행렬 R을 2개의 Feature Matrix P와 Q로 분해
* P : k 차원의 벡터를 사용자 수 만큼 모아놓은 행렬 = 사용자의 특성 벡터
* Q : 영화의 특성 벡터
* 해당 벡터의 내적으로 사용자의 선호도를 구한다.
* 벡터를 잘 만드는 기준은 유저 i의 벡터와 아이템의 j의 벡터를 내적했을 때 유저 i가 아이템 j에 대해 평가한 수치와 비슷한지 확인
* 선호/비선호로 나뉘기 때문에 유저와 유저가 좋아하는 노래의 곱한 수치는 1에 가까워져야 한다.

## CSR(Compressed Sparse Row) Matrix

* 0이 아닌 유효 데이터의 좌표 정보로만 구성
* 메모리 사용량을 최소화하면서 Sparse 한 Matrix와 동일한 행렬을 표현할 수 있게 한다.
* CSR Matrix : data, indices, indptr 행렬로 압축하여 표현
* data : 0이 아닌 원소를 차례대로 기입
* indices : 각 요소가 어느 열에 있는지를 표현한 index
* indptr : 최초시작행번호, 시작행에서 데이타의 개수, 두번째 행의 데이터의 누적개수...마지막 행의 데이터의 누적 개수

* [scipy.sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)

In [14]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Row format>

## MF 모델 학습하기

### 학습

* implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.
* 이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다.
* Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않습니다.
* 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.

In [15]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

* Implicit AlternatingLeastSquares 모델의 선언

In [16]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

* als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)

* 하지만 이는 0.4.8 버전에서만 해당되고 신버전인 0.6.0에서는 transpose 할 필요가 없습니다.
* user X item

In [17]:
# csr_data_transpose = csr_data.T
# csr_data_transpose

* 모델 훈련

In [18]:
als_model.fit(csr_data)

  0%|          | 0/15 [00:00<?, ?it/s]

### 살펴보기

In [19]:
als_model

* Zimin 벡터와 blacked eyed peas 벡터를 어떻게 만들고 있는지
* 두 벡터를 곱하면 어떻게 되는지

In [20]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

In [21]:
zimin_vector

array([-0.7200203 ,  0.18979211, -0.03573835,  0.48045513,  0.46122485,
       -0.05038334, -0.588919  ,  1.2002501 , -0.27211913,  0.0098492 ,
        0.05546581,  0.815551  ,  0.6768949 ,  0.18631971, -0.5263393 ,
        0.69343615, -0.8123301 ,  0.21940947, -0.66332984, -0.9932918 ,
       -1.2320694 ,  0.6114599 ,  0.2824139 , -1.2281735 , -0.15176488,
       -0.07289906, -0.91242886, -0.4575416 ,  0.75882024, -0.930476  ,
       -0.13526635,  0.55520475, -0.45148814, -0.34611407, -0.04469076,
       -1.1738919 , -0.62270206,  0.5563129 ,  0.8448182 ,  0.5989601 ,
       -0.05246093, -0.3991524 ,  0.79394245, -0.5773073 ,  0.08808918,
       -0.35008383, -1.0301887 , -0.2659286 ,  0.27394497,  0.36987868,
       -0.3454218 , -0.17795381,  0.21347508, -0.28581133, -0.42724797,
        0.96106976, -1.331309  ,  0.24690041, -0.16871071,  0.17230576,
        0.60406   ,  0.05807182, -0.52527285, -0.2724236 ,  1.0666229 ,
        0.634138  ,  0.05075198,  0.14366777,  1.405964  , -1.02

In [22]:
black_eyed_peas_vector

array([ 0.00627707,  0.01115307,  0.01245579,  0.00704328,  0.00368194,
       -0.00258279, -0.00256922,  0.01560322,  0.00874362,  0.00194606,
        0.00915628,  0.01610529,  0.01675873,  0.00977897,  0.01195157,
        0.00937805,  0.00171927,  0.00043923, -0.00526564,  0.00441086,
       -0.00805358,  0.01869972, -0.00299414, -0.01726981, -0.00173491,
        0.01156667,  0.00103871, -0.00992885,  0.00624374,  0.00974588,
        0.00241598,  0.01950083,  0.00262619,  0.00066521,  0.02331159,
       -0.00701431,  0.00923102, -0.00339496,  0.01326437,  0.0066893 ,
        0.00491552, -0.00370508,  0.01657542, -0.0130049 ,  0.00195481,
        0.00675307,  0.00971233,  0.00084021,  0.01628168, -0.00077194,
        0.00517276, -0.00587565,  0.01596999, -0.00649978, -0.00118863,
        0.02574253, -0.01394115,  0.00074796, -0.00277773, -0.00234112,
        0.00381129,  0.01267835, -0.00888735, -0.00534133,  0.025982  ,
        0.00895749,  0.02077922,  0.0105363 ,  0.02053885,  0.00

In [23]:
np.dot(zimin_vector, black_eyed_peas_vector)

0.5021421

In [24]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.2872602

## 비슷한 아티스트 찾기 + 유저에게 추천하기

### 비슷한 아티스트 찾기

* AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통해 비슷한 아티스트를 찾습니다.
* 처음으로 coldplay를 찾아봅시다.

In [25]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist_idx, similar_artist = als_model.similar_items(artist_id, N=15)
print(similar_artist_idx)
print(similar_artist)
tuple_similar_artist = [(similar_artist_idx[i], similar_artist[i]) for i in range(15)]

[  62  277   28    5  247  473  217  490  418  694  910  268 1018  782
   75]
[1.         0.9872002  0.983138   0.9783751  0.97063065 0.9700052
 0.9670455  0.96576846 0.96027297 0.9563995  0.95121473 0.95014554
 0.946873   0.93734753 0.93580264]


* Tuple로 반환하고 있습니다. 아티스트 id를 다시 아티스트 이름으로 매핑
* artist_to_idx를 뒤집어, index로부터 artist 이름을 얻는 dict을 생성

In [26]:
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in tuple_similar_artist]

['coldplay',
 'muse',
 'the killers',
 'red hot chili peppers',
 'the beatles',
 'placebo',
 'radiohead',
 'oasis',
 'u2',
 'foo fighters',
 'nirvana',
 'pink floyd',
 'the smashing pumpkins',
 'the white stripes',
 'queen']

* 함수화

In [33]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist_idx, similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i] for i in similar_artist_idx]
    return similar_artist

In [39]:
for i  in similar_artist_idx:
    print(idx_to_artist[i])

coldplay
muse
the killers
red hot chili peppers
the beatles
placebo
radiohead
oasis
u2
foo fighters
nirvana
pink floyd
the smashing pumpkins
the white stripes
queen


In [40]:
get_similar_artist('2pac')

['2pac',
 'notorious b.i.g.',
 'nas',
 'the game',
 'jay-z',
 'ludacris',
 'bone thugs-n-harmony',
 'snoop dogg',
 'ice cube',
 'gang starr']

In [36]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'rihanna',
 'katy perry',
 'the pussycat dolls',
 'beyoncé',
 'christina aguilera',
 'justin timberlake',
 'leona lewis',
 'kelly clarkson']

### 유저에게 아티스트 추천하기

* AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 아티스트를 추천받습니다.
* filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [41]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended_idx, artist_recommended = als_model.recommend(user, csr_data[user], N=15, filter_already_liked_items=True)
tuple_artist_recommended = [(artist_recommended_idx[i], artist_recommended[i]) for i in range(15)]
tuple_artist_recommended

[(350, 0.46499622),
 (369, 0.45691496),
 (550, 0.45192122),
 (1800, 0.4433067),
 (2249, 0.4423669),
 (627, 0.42191452),
 (621, 0.42023432),
 (274, 0.4185737),
 (391, 0.41479212),
 (564, 0.4097677),
 (355, 0.39925665),
 (354, 0.3897612),
 (724, 0.3860433),
 (409, 0.37312168),
 (358, 0.36723334)]

In [46]:
[idx_to_artist[i] for i, _ in tuple_artist_recommended]

['rihanna',
 'justin timberlake',
 'britney spears',
 'lady gaga',
 'katy perry',
 'maroon 5',
 'alicia keys',
 'michael jackson',
 'christina aguilera',
 'kanye west',
 'madonna',
 'nelly furtado',
 'lily allen',
 'amy winehouse',
 'kelly clarkson']

* 추천에 기여한 정도 확인

In [47]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [49]:
[(idx_to_artist[i], j) for i, j in explain[1]]

[('beyoncé', 0.2296966857217802),
 ('black eyed peas', 0.1520633420404597),
 ('jason mraz', 0.04913960896626038),
 ('coldplay', 0.03456913312280204),
 ('maroon5', 0.0006103158131281585)]

*  beyonce와 black eyed peas 가 가장 크게 기여했네요.